In [1]:
import pandas as pd
import csv
import numpy as np
import os

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def make_df(i):
  if i<10:
    f_anno_path = '/content/drive/MyDrive/ETRI/KEMDy19/annotation/Session0%d_F_res.csv' %i
    m_anno_path = '/content/drive/MyDrive/ETRI/KEMDy19/annotation/Session0%d_M_res.csv' %i
  else:
    f_anno_path = '/content/drive/MyDrive/ETRI/KEMDy19/annotation/Session%d_F_res.csv' %i
    m_anno_path = '/content/drive/MyDrive/ETRI/KEMDy19/annotation/Session%d_M_res.csv' %i



  f_df = pd.DataFrame()
  m_df = pd.DataFrame()

  f_df = pd.read_csv(f_anno_path, usecols=[9,10])
  f_df.rename(columns={'Segment ID':'Seg', 'Total Evaluation':'Emotion'}, inplace=True)
  f_df = f_df.drop([0])

  m_df = pd.read_csv(m_anno_path, usecols=[9,10])
  m_df.rename(columns={'Segment ID':'Seg', 'Total Evaluation':'Emotion'}, inplace=True)
  m_df = m_df.drop([0])

  f_df['Role'] = 'Speaker'
  m_df['Role'] = 'Speaker'

  f_listener_index = []
  m_listener_index = []

  f_listener_index = f_df[f_df['Seg'].str.contains('M')].index 
  m_listener_index = m_df[m_df['Seg'].str.contains('F')].index

  for i in f_listener_index:
    f_df['Role'][i]='Listener'

  for j in m_listener_index:
    m_df['Role'][j]='Listener'

  sp_df = pd.DataFrame()
  li_df = pd.DataFrame()

  f_sp = pd.DataFrame()
  f_li = pd.DataFrame()
  m_sp = pd.DataFrame()
  m_li = pd.DataFrame()

  sp_df = pd.concat([f_df[f_df['Role']=='Speaker'], m_df[m_df['Role']=='Speaker']])
  li_df = pd.concat([f_df[f_df['Role']=='Listener'], m_df[m_df['Role']=='Listener']])

  def Select(feat):
    if ";" in feat:
      semicolon = feat.find(";")
      return feat[:semicolon]
    else:
      return feat

  sp_df['Emotion'] = sp_df['Emotion'].apply(lambda feat:Select(feat))
  li_df['Emotion'] = li_df['Emotion'].apply(lambda feat:Select(feat))


  return sp_df,li_df

In [7]:
sp_df_01, li_df_01 = make_df(1)
sp_df_02, li_df_02 = make_df(2)
sp_df_03, li_df_03 = make_df(3)
sp_df_04, li_df_04 = make_df(4)
sp_df_05, li_df_05 = make_df(5)
sp_df_06, li_df_06 = make_df(6)
sp_df_07, li_df_07 = make_df(7)
sp_df_08, li_df_08 = make_df(8)
sp_df_09, li_df_09 = make_df(9)
sp_df_10, li_df_10 = make_df(10)
sp_df_11, li_df_11 = make_df(11)
sp_df_12, li_df_12 = make_df(12)
sp_df_13, li_df_13 = make_df(13)
sp_df_14, li_df_14 = make_df(14)
sp_df_15, li_df_15 = make_df(15)
sp_df_16, li_df_16 = make_df(16)
sp_df_17, li_df_17 = make_df(17)
sp_df_18, li_df_18 = make_df(18)
sp_df_19, li_df_19 = make_df(19)
sp_df_20, li_df_20 = make_df(20)

In [8]:
from sklearn.model_selection import train_test_split

def make_train_test(num):
  sp_train = pd.DataFrame()
  sp_test = pd.DataFrame()
  li_train = pd.DataFrame()
  li_test = pd.DataFrame()
  for i in range(num):
    sp_df, li_df = make_df(i+1)

    sp_train_temp, sp_test_temp = train_test_split(sp_df, test_size=0.2, random_state=42)
    li_train_temp, li_test_temp = train_test_split(li_df, test_size=0.2, random_state=42)

    sp_train = pd.concat([sp_train, sp_train_temp])
    sp_test = pd.concat([sp_test, sp_test_temp])
    li_train = pd.concat([li_train, li_train_temp])
    li_test = pd.concat([li_test, li_test_temp])

  return sp_train, sp_test, li_train, li_test

In [9]:
sp_train, sp_test, li_train, li_test = make_train_test(20)

In [12]:
sp_train.reset_index(drop=True, inplace=True)
sp_test.reset_index(drop=True, inplace=True)
li_train.reset_index(drop=True, inplace=True)
li_test.reset_index(drop=True, inplace=True)

In [14]:
def make_path(df):
  df['wav_path'] = 0
  df['txt_path'] = 0
  for i in range(len(df)):
    df['wav_path'][i] = '/content/drive/MyDrive/ETRI/KEMDy19/wav/'+df['Seg'][i][:4]+'ion'+df['Seg'][i][4:6]+'/'+df['Seg'][i][:-5]+'/'+df['Seg'][i]+'.wav'
    df['txt_path'][i] = '/content/drive/MyDrive/ETRI/KEMDy19/wav/'+df['Seg'][i][:4]+'ion'+df['Seg'][i][4:6]+'/'+df['Seg'][i][:-5]+'/'+df['Seg'][i]+'.txt'

  return df

In [15]:
sp_train = make_path(sp_train)
sp_test = make_path(sp_test)
li_train = make_path(li_train)
li_test = make_path(li_test)

<ipython-input-14-a110b6b83a0b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['wav_path'][i] = '/content/drive/MyDrive/ETRI/KEMDy19/wav/'+df['Seg'][i][:4]+'ion'+df['Seg'][i][4:6]+'/'+df['Seg'][i][:-5]+'/'+df['Seg'][i]+'.wav'
<ipython-input-14-a110b6b83a0b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['txt_path'][i] = '/content/drive/MyDrive/ETRI/KEMDy19/wav/'+df['Seg'][i][:4]+'ion'+df['Seg'][i][4:6]+'/'+df['Seg'][i][:-5]+'/'+df['Seg'][i]+'.txt'
<ipython-input-14-a110b6b83a0b>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation:

In [ ]:
sp_train.to_csv("/content/drive/MyDrive/ETRI/sp_train.csv", mode='w')
sp_test.to_csv("/content/drive/MyDrive/ETRI/sp_test.csv", mode='w')
li_train.to_csv("/content/drive/MyDrive/ETRI/li_train.csv", mode='w')
li_test.to_csv("/content/drive/MyDrive/ETRI/li_test.csv", mode='w')